In [ ]:
# STEP 1: Install Dependencies
!pip install -q datasets scikit-learn ibm-cos-sdk ibm-watson-machine-learning==1.0.312
!pip install --upgrade pandas==2.1.


In [ ]:

# STEP 2: Setup and Load Data
import os, getpass, types, time
import pandas as pd
from botocore.client import Config
import ibm_boto3

# IBM WML Credentials
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Enter your IBM Cloud API key: ")
}

# Project ID
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your IBM Cloud project_id: ")

# IBM Cloud Object Storage
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rHRXQIpWLwXJr1IcHTYnz_JA2pjArLkSXINRgSFQmiay',  # Replace with actual COS API key
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucket-r33nj527e8ujf21'  # Replace with your bucket name
train_key = 'ecommerce_reviews_train.csv'
test_key = 'ecommerce_reviews_test.csv'

def __iter__(self): return 0

# Load Training Data
train_body = cos_client.get_object(Bucket=bucket, Key=train_key)['Body']
if not hasattr(train_body, "__iter__"): train_body.__iter__ = types.MethodType(__iter__, train_body)
train_data = pd.read_csv(train_body)

# Load Test Data
test_body = cos_client.get_object(Bucket=bucket, Key=test_key)['Body']
if not hasattr(test_body, "__iter__"): test_body.__iter__ = types.MethodType(__iter__, test_body)
test_data = pd.read_csv(test_body)

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

In [ ]:
# STEP 3: Initialize Watsonx Foundation Model
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

model_id = ModelTypes.FLAN_UL2

parameters = {
    GenParams.MAX_NEW_TOKENS: 10
}

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)


In [ ]:
# STEP 4: Predict Categories Using Prompt Engineering
instruction = """
Classify the product review into one of the following categories: Electronics, Clothing, or Home Appliances.\n
review: This phone has amazing battery life and is super fast.\n
category: Electronics\n\n
review: The jeans fit perfectly and feel comfortable.\n
category: Clothing\n\n
review: The blender works great and makes smoothies quickly.\n
category: Home Appliances\n\n
"""

results = []
reviews = list(test_data[['Product ID', 'Review Text', 'Product Category']].values)  # Load Product ID & Actual Category

for i, (product_id, review, actual_category) in enumerate(reviews):
    prompt = instruction + f"review: {review}\ncategory:"
    response = model.generate_text(prompt=prompt)

    predicted_category = response.strip()  # Directly use response as the predicted category
    
    print(f" Product ID: {product_id} | Review: {review} | Predicted Category: {predicted_category} | Actual Category: {actual_category}\n")
    time.sleep(0.5)  # To avoid rate limits